<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamA/blob/main/Qry_cond_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas as pd
import numpy as np
import sqlite3
# import pandas_profiling
# from datetime import datetime
# import re
import seaborn as sns
import matplotlib.pyplot as plt
# from pandas_profiling import ProfileReport
# will make plot outputs appear and stored within the notebook.
%matplotlib inline
from sqlite3 import Error

from google.colab import drive
drive.mount('/content/drive')

# Path
material_path = "/content/drive/MyDrive/M3_DM_GrpA"

# path of datawarehouse
DB_DWH_PATH = material_path + f"/DWH_MAPCOV.db"

dwh_db = sqlite3.connect(DB_DWH_PATH)
dwh_cursor = dwh_db.cursor()

Mounted at /content/drive


Conditions: Filter nach Conditions, die parallel zu COVID aufgetreten sind.  
Erster Part enthält den allgemeien Filter in der Conditions Tabelle nach Start und Stop von COVID-19.  
Danach Merge mit der Conditions Tabelle OHNE 'COVID-19'. Das Ergebnis ist, dass hinter jedem Condition das Start/Stop Date von COVID steht. Nun kann eine Abfrage (Query) geschrieben werden, die die Datumsangaben vergleicht und filtert.

In [4]:
df_conditions_filter = pd.read_sql_query("SELECT START AS stdt_covid, STOP AS endt_covid, PATIENT FROM conditions_INFO WHERE DESCRIPTION = 'COVID-19'", dwh_db)
print("Patients with Covid:", df_conditions_filter.shape)
df_cond_other = pd.read_sql_query("SELECT START, STOP, PATIENT, DESCRIPTION FROM conditions_info WHERE DESCRIPTION NOT LIKE '%COVID%'", dwh_db)
print("Number of other conitons than Covid:",df_cond_other.shape)
df_cond_covid = pd.merge(df_cond_other, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_cond_covid.shape)
df_cond_covid_qry = df_cond_covid.query('(STOP >= stdt_covid or STOP == 0) and (START <= endt_covid or endt_covid == 0)')
print("Filter for cond within the same time as COVID:",df_cond_covid_qry.shape)
df_cond_covid_qry.head()

# Alternativ: Lists the count for all 'DESCRIPTION'
# df_cond_covid_qry.groupby(["DESCRIPTION"]).size()

Patients with Covid: (865, 3)
Number of other conitons than Covid: (4605, 4)
Merge: (4361, 6)
Filter for cond within the same time as COVID: (3729, 6)


DESCRIPTION
Acute deep venous thrombosis (disorder)       27
Acute pulmonary embolism (disorder)           24
Acute respiratory failure (disorder)          41
Chill (finding)                               93
Cough (finding)                              553
Diarrhea symptom (finding)                    24
Dyspnea (finding)                            153
Fatigue (finding)                            326
Fever (finding)                              750
Headache (finding)                           108
Heart failure (disorder)                       1
Hemoptysis (finding)                          11
Injury of kidney (disorder)                    1
Joint pain (finding)                         135
Loss of taste (finding)                      426
Muscle pain (finding)                        135
Nasal congestion (finding)                    46
Nausea (finding)                              44
Passive conjunctival congestion (finding)      6
Pneumonia (disorder)                         105
Respirat